In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("NORD_Task") \
      .getOrCreate() 
sc = spark.sparkContext




hadoop_conf=sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")

S3_BUCKET = 's3-nord-challenge-data'
S3_REGION = 'eu-central-1'
hadoop_conf.set("fs.s3a.endpoint", f"s3.{S3_REGION}.amazonaws.com")



#number of files to process - will be read as input
N = 100

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.amazonaws#aws-java-sdk added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6d8dbff1-c103-4c39-a243-8000651782f3;1.0
	confs: [default]
	found com.amazonaws#aws-java-sdk;1.11.901 in central
	found com.amazonaws#aws-java-sdk-appregistry;1.11.901 in central
	found com.amazonaws#aws-java-sdk-core;1.11.901 in central
	found commons-logging#commons-logging;1.1.3 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-codec#commons-codec;1.11 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found com.fasterxml.jackson.core#jackson-databind;2.6.7.3 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.6.0 in central
	found com.fasterxml.jacks

	found com.amazonaws#aws-java-sdk-alexaforbusiness;1.11.901 in central
	found com.amazonaws#aws-java-sdk-resourcegroups;1.11.901 in central
	found com.amazonaws#aws-java-sdk-comprehend;1.11.901 in central
	found com.amazonaws#aws-java-sdk-translate;1.11.901 in central
	found com.amazonaws#aws-java-sdk-sagemaker;1.11.901 in central
	found com.amazonaws#aws-java-sdk-iotjobsdataplane;1.11.901 in central
	found com.amazonaws#aws-java-sdk-sagemakerruntime;1.11.901 in central
	found com.amazonaws#aws-java-sdk-kinesisvideo;1.11.901 in central
	found io.netty#netty-codec-http;4.1.48.Final in central
	found io.netty#netty-common;4.1.48.Final in central
	found io.netty#netty-buffer;4.1.48.Final in central
	found io.netty#netty-transport;4.1.48.Final in central
	found io.netty#netty-resolver;4.1.48.Final in central
	found io.netty#netty-codec;4.1.48.Final in central
	found io.netty#netty-handler;4.1.48.Final in central
	found com.amazonaws#aws-java-sdk-appsync;1.11.901 in central
	found com.amazo

	0 artifacts copied, 254 already retrieved (0kB/43ms)
22/12/11 01:42:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Note 
In general I can see 2 approaches to load files data:
   - Approach 1.
     - `spark.read.format('binaryFile').option("pathGlobFilter","<path-glob>").load(<s3-bucket>)`. This solution would read all files with metadata into single DataFrame (path, mod time,  length, content)
     - parse content of file in apropriate resulted dataframe transformation
   - The advantage of it is that you receive parallelized DataFrame, content of file would be read in lazy way during processing each file. So in theory on a big enough spark cluster spark should take care of distributing and performance for you. The problem seems to be when you have to work with pretty read milions of files with unknown file size. You may end up huge memory and performance issues. This problem is shown e.g. in [this blog article](https://wrightturn.wordpress.com/2015/07/22/getting-spark-data-from-aws-s3-using-boto-and-pyspark/). Although it's pretty old I did not find any more recent solution to the issue. It also describes second approach.
   - Approach 2.
       - list all objects you're interested files in s3 bucket into some collection (but without parallelizing it)
       - create parallelized dataframe based on the given collection
       - read and process file content as part of transformations
   - The bottleneck might that you have to iterate over millions of files so the size of the collection to be processed (on one node) might be huge. 
   
As I am not able to test on a large set of data and big enough spark cluster which approach is more efficient. I am going to use approach described in [mentioned article](https://wrightturn.wordpress.com/2015/07/22/getting-spark-data-from-aws-s3-using-boto-and-pyspark). However instead of using boto3 for listing all objects I will use [`hadoop.fs.path.getFilesystem.globStatus`](https://stackoverflow.com/a/67050173/2018369) because boto3 [seems to be not the most effective way](https://stackoverflow.com/q/69920805/2018369) to get file list.

I was also considering one more approach, which however I could not find any good way to implement. So my idea was to create a dataframe similar to the one created by `spark.read.format('binaryFile').option("pathGlobFilter","<path-glob>").load(<s3-bucket>)`, but which contain only prefix of file (first 1024 or 2048 bytes). This way we could have a Dataframe(path, mod time,  length, PE headers), we could process the header of file to get all required PE metadata apart from imports/expors and in next step we could load apropriate sections of file to get imports/exports.


In [2]:
clean_path = '/0/*.???'
malware_path = '/1/*.???'



cleanPath = sc._jvm.org.apache.hadoop.fs.Path(f's3a://{S3_BUCKET}{clean_path}')
cFs = cleanPath.getFileSystem(hadoop_conf)
clean_files = cFs.globStatus(cleanPath)

malwarePath = sc._jvm.org.apache.hadoop.fs.Path(f's3a://{S3_BUCKET}{malware_path}')
mFs = malwarePath.getFileSystem(hadoop_conf)
malware_files = mFs.globStatus(malwarePath)

22/12/11 01:43:00 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [3]:
import random
print(len(malware_files))
files_to_process = random.sample(clean_files, int(N/2))+ random.sample(malware_files, int(N/2))
print(len(files_to_process))

malware_files

14652
100


JavaObject id=o38

In [4]:
# put files into dataFrame

from pyspark.sql.types import StructType,StructField, StringType, IntegerType
schema = StructType([       
    StructField('path', StringType(), True),
    StructField('size', IntegerType(), True),
    StructField('type', StringType(), True)
])
data = [(f.getPath().toUri().getRawPath(), f.getLen(), f.getPath().getName().split('.')[-1]) for f in files_to_process]
# data = [(f.getPath().toUri().getRawPath(), f.getLen(), f.getPath().getName().split('.')[-1]) for f in clean_files]
# make sure we don't have duplicates
filesDF= spark.createDataFrame(data=data, schema = schema).distinct()
filesDF.orderBy("size").show()

+--------------------+-----+----+
|                path| size|type|
+--------------------+-----+----+
|/0/ENrSk90bJebhOq...|  243| exe|
|/0/8H7W3N6HeHfFQw...|  253| dll|
|/0/5fyjRfE9HxFOlz...| 4122| dll|
|/1/DFxuoEsbYCzcvm...| 5984| dll|
|/0/cg8ddt6kkgwdrL...| 6030| dll|
|/0/oSKfjC29VEr1R4...| 7680| DLL|
|/0/TyL4rbyRfKi8Ll...| 8192| DLL|
|/0/pTSl2ofrq2SQpk...| 8192| DLL|
|/0/hdnTbip7oFQTp6...|13824| dll|
|/0/vTN1eOe2VJD0lJ...|13824| dll|
|/0/cKIKYKxfwFmFKl...|13824| dll|
|/0/Idca5TgntwoGGX...|14226| dll|
|/0/FOwWabB3nnfDok...|14848| DLL|
|/0/eIloYq7cHIaUFi...|17680| DLL|
|/0/1S2mvr347Owlf8...|20792| dll|
|/0/6EB1y6698C91G4...|21232| dll|
|/0/9IprmZvYdNi08K...|22116| dll|
|/0/doB5Z7qwjQxpCi...|25592| dll|
|/0/XCxdoGLWJjvD89...|26224| dll|
|/0/tNiJNBTxNrLhif...|30256| dll|
+--------------------+-----+----+
only showing top 20 rows



In [5]:
filesDF.orderBy('size').show(50,False)

+---------------------------------------+------+----+
|path                                   |size  |type|
+---------------------------------------+------+----+
|/0/ENrSk90bJebhOqYUNlgD3iuB8c7wFkKI.exe|243   |exe |
|/0/8H7W3N6HeHfFQw34ZAMZV1a3UwgtM33c.dll|253   |dll |
|/0/5fyjRfE9HxFOlzBPGAVS1wyqi58YCBrw.dll|4122  |dll |
|/1/DFxuoEsbYCzcvmJN5X1eH2NhtIInssFd.dll|5984  |dll |
|/0/cg8ddt6kkgwdrLA7E6LKE34nEZ4Yh18v.dll|6030  |dll |
|/0/oSKfjC29VEr1R4q3lZTYoYHZNCptJi1f.DLL|7680  |DLL |
|/0/pTSl2ofrq2SQpkgaaOFXQthIs4pcOe13.DLL|8192  |DLL |
|/0/TyL4rbyRfKi8LlKj7adF7M21FhEXDiZx.DLL|8192  |DLL |
|/0/vTN1eOe2VJD0lJiXygzrmmHI28vl0j4l.dll|13824 |dll |
|/0/cKIKYKxfwFmFKlDr8mankem1nGASB8ss.dll|13824 |dll |
|/0/hdnTbip7oFQTp6tWkJ07gVqSowCK0l5N.dll|13824 |dll |
|/0/Idca5TgntwoGGXJf1ve0HSovCWLIapKJ.dll|14226 |dll |
|/0/FOwWabB3nnfDokCidtvmA78SiSVGgC8I.DLL|14848 |DLL |
|/0/eIloYq7cHIaUFiBvr0jWJD4oeZQ7brEh.DLL|17680 |DLL |
|/0/1S2mvr347Owlf8QgygSiG0JhUaW0VzYJ.dll|20792 |dll |
|/0/6EB1y6698C91G4RzkTA8tFQb

# Database Notes
I was considering SQL and NoSQL (key/value store) to store files info.

### SQL Database - MySQL
For now I decided to use simple solution  - MySQL database. This is pretty simple to replace with different SQL database or even with totally different solution in case of efficiency issues.

The architecture is rather not complicated. All **distinct** file records are processed and stored in one table with following schema
 `path Varchar primary key, size Int, type Varchar, architecture Varchar default NULL, imports Int default NULL, exports Int default NULL, INDEX(size, type));`
At the whole table is loaded into DataFrame. It is substracted from task files DF to ensure already processed files are skipped. And after processing transformed DF is appended to existing table in MySQL.

Although number of files processed can reach (hundred of) millions [MySQL should handle it properly](https://dba.stackexchange.com/questions/20335/can-mysql-reasonably-perform-queries-on-billions-of-rows) with proper indexes. If there are billions of rows in DB we might start [encontering problems](https://stackoverflow.com/questions/38346613/mysql-and-a-table-with-100-millions-of-rows)
In case of performance issue we may think about using different Database type.

### NoSQL solutions
I was considering also NoSQL database which very often perform better in distributed environment and in most cases scale horizontally much easier than classical SQL DB. For this task I consider key/value store as a good solution.

#### Aerospike
Aerospike was considered as it promises high efficiency, distributed (based on shared nothing architecture) database for storing key/value pairs. In commercial version it support pyspark distributed operations, direct import to RDDs etc. So if required it might give very good performance.

#### Redis
Open source, in-memory data store used as a database, cache, streaming engine, and message broker.


### "hybrid" approach
The issue with in-memory key/value store is that it does not provide (by default) persistence of data.
This can be achieved both in Redis and Aerospike of course but not by default.

My idea was to provide hybrid solution in which processed files data is stored in classical SQL database but apart from that it is also imported into key/value store. In that case there is no need to load all existing entries into DataFrame prior to processing new entries just to make sure some files weren't already processed. Instead, I would filter out from `filesDF` entries that exists in key/value store during transformations. As a last steps I would need to save (append) `filesDB` not only to SQL database but also to key/value store




In [14]:
# DB Settings
jdbc_url = 'jdbc:mysql://db/nord_files'
table = "files_info"
username = "root"
password = "password"
driver = "com.mysql.cj.jdbc.Driver"


In [15]:
# get files already processed from database

db_files_info = spark.read.format("jdbc")\
    .options(url =jdbc_url,driver=driver, dbtable=table,user=username,password=password).load().select(["path", "size", "type"])


print(db_files_info.columns)
db_files_info.show()





['path', 'size', 'type']
+----+----+----+
|path|size|type|
+----+----+----+
+----+----+----+



In [16]:
# Remove files that exists in DB from list of files to process
filesDF = filesDF.subtract(db_files_info)
filesDF.show()

+--------------------+-------+----+
|                path|   size|type|
+--------------------+-------+----+
|/0/lYa3OJQ7Jc7gV3...|1514496| dll|
|/0/EqMIUXDSZByy84...| 350208| dll|
|/1/PNY3tSTnJxWNqN...| 957328| exe|
|/1/72F9skoss7i36A...| 423936| exe|
|/0/9IprmZvYdNi08K...|  22116| dll|
|/1/u1xUr4O62n86yu...| 118784| exe|
|/0/1S2mvr347Owlf8...|  20792| dll|
|/0/FOwWabB3nnfDok...|  14848| DLL|
|/1/kios4K0qRVRcYz...| 347136| dll|
|/0/cg8ddt6kkgwdrL...|   6030| dll|
|/0/hdnTbip7oFQTp6...|  13824| dll|
|/1/JdPfYEtwLCLLGa...| 345088| exe|
|/1/6MonM6f6numehp...| 475648| exe|
|/0/5fyjRfE9HxFOlz...|   4122| dll|
|/0/Q4xQ29rF2Jfw8k...|  41472| dll|
|/0/c0Hy3S7ZwlFnJg...| 521216| dll|
|/0/drA5FACeAECwP4...| 506368| dll|
|/1/8k9wb4KSIy3uAA...| 723456| exe|
|/1/70vUPKHDtlpR5x...| 352256| exe|
|/0/hZEtUAgvlz1y68...| 453787| dll|
+--------------------+-------+----+
only showing top 20 rows



In [ ]:
# process files 

from s3file_reader import S3FileReader
from pe_parser import PeParser


schema = StructType(filesDF.schema.fields+[
    StructField('architecture', StringType(), True),
    StructField('imports', IntegerType(), True),
    StructField('exports',IntegerType(), True)
])


def parse_f(x):
    """
    This is a function that is applied using rdd map to each file in Dataframe.
    It parses the given file from s3 and returns meta (architecture, imports, exports) for it.
    In case of error (e.g. corrupted file) it returns None, -1, -1
    """
    reader = S3FileReader(S3_BUCKET, S3_REGION)
    path = x['path'].lstrip("/")
    size = x['size']
    parser = PeParser(path, reader.get_file_stream(path), size)
    try:
        return (*x, *parser.get_short_meta())
    except ValueError:
        return (*x, None, -1, -1)




parsed=filesDF.rdd.map(parse_f)

parsedDF = parsed.toDF(schema)
parsedDF.show()


<botocore.response.StreamingBody object at 0x7f1ceef85630>          (0 + 1) / 1]


In [ ]:
# Store result to DB
parsedDF.write.format('jdbc').options(
    url=jdbc_url, driver=driver,dbtable=table, user=username, password=password
).mode('append').save()



<botocore.response.StreamingBody object at 0x7f1ceef85630>          (0 + 1) / 1]
